<a href="https://colab.research.google.com/github/tashfi04/swe-427-ML-assignments/blob/main/AI_Final_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [3]:
!gdown 18KwSR9aVTZRNaOVF76VE9USSEkqnYzzQ

Downloading...
From: https://drive.google.com/uc?id=18KwSR9aVTZRNaOVF76VE9USSEkqnYzzQ
To: /content/bank-data.zip
100% 516k/516k [00:00<00:00, 36.9MB/s]


In [4]:
!unzip 'bank-data.zip'

Archive:  bank-data.zip
   creating: bank-data/
  inflating: __MACOSX/._bank-data    
  inflating: bank-data/.DS_Store     
  inflating: __MACOSX/bank-data/._.DS_Store  
  inflating: bank-data/bank-full.csv  
  inflating: __MACOSX/bank-data/._bank-full.csv  


# Read CSV

In [5]:
df = pd.read_csv("bank-data/bank-full.csv", index_col=None)
df

,"age;""job"";""marital"";""education"";""default"";""balance"";""housing"";""loan"";""contact"";""day"";""month"";""duration"";""campaign"";""pdays"";""previous"";""poutcome"";""y"""
0,"58;""management"";""married"";""tertiary"";""no"";2143..."
1,"44;""technician"";""single"";""secondary"";""no"";29;""..."
2,"33;""entrepreneur"";""married"";""secondary"";""no"";2..."
3,"47;""blue-collar"";""married"";""unknown"";""no"";1506..."
4,"33;""unknown"";""single"";""unknown"";""no"";1;""no"";""n..."
...,...
45206,"51;""technician"";""married"";""tertiary"";""no"";825;..."
45207,"71;""retired"";""divorced"";""primary"";""no"";1729;""n..."
45208,"72;""retired"";""married"";""secondary"";""no"";5715;""..."
45209,"57;""blue-collar"";""married"";""secondary"";""no"";66..."


# Remove unnecessary columns

In [6]:
df = pd.read_csv('bank-data/bank-full.csv', delimiter=';').drop([
    'default', 'contact', 'day', 'month', 'pdays', 'previous', 'poutcome'
], axis=1)

In [7]:
df

,age,job,marital,education,balance,housing,loan,duration,campaign,y
0,58,management,married,tertiary,2143,yes,no,261,1,no
1,44,technician,single,secondary,29,yes,no,151,1,no
2,33,entrepreneur,married,secondary,2,yes,yes,76,1,no
3,47,blue-collar,married,unknown,1506,yes,no,92,1,no
4,33,unknown,single,unknown,1,no,no,198,1,no
...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,825,no,no,977,3,yes
45207,71,retired,divorced,primary,1729,no,no,456,2,yes
45208,72,retired,married,secondary,5715,no,no,1127,5,yes
45209,57,blue-collar,married,secondary,668,no,no,508,4,no


# Encode data

In [8]:
print(df['job'].unique(), df['marital'].unique(), df['education'].unique(), df['housing'].unique(), df['y'].unique())

['management' 'technician' 'entrepreneur' 'blue-collar' 'unknown'
 'retired' 'admin.' 'services' 'self-employed' 'unemployed' 'housemaid'
 'student'] ['married' 'single' 'divorced'] ['tertiary' 'secondary' 'unknown' 'primary'] ['yes' 'no'] ['no' 'yes']


In [9]:
le = LabelEncoder()

df['job'] = le.fit_transform(df['job'])
df['marital'] = le.fit_transform(df['marital'])
df['education'] = le.fit_transform(df['education'])
df['housing'] = le.fit_transform(df['housing'])
df['y'] = le.fit_transform(df['y'])
df['loan'] = le.fit_transform(df['loan'])

In [10]:
print(df['job'].unique(), df['marital'].unique(), df['education'].unique(), df['housing'].unique(), df['y'].unique())

[ 4  9  2  1 11  5  0  7  6 10  3  8] [1 2 0] [2 1 3 0] [1 0] [0 1]


In [11]:
df

,age,job,marital,education,balance,housing,loan,duration,campaign,y
0,58,4,1,2,2143,1,0,261,1,0
1,44,9,2,1,29,1,0,151,1,0
2,33,2,1,1,2,1,1,76,1,0
3,47,1,1,3,1506,1,0,92,1,0
4,33,11,2,3,1,0,0,198,1,0
...,...,...,...,...,...,...,...,...,...,...
45206,51,9,1,2,825,0,0,977,3,1
45207,71,5,0,0,1729,0,0,456,2,1
45208,72,5,1,1,5715,0,0,1127,5,1
45209,57,1,1,1,668,0,0,508,4,0


# Split data into training & testing (80:20)

In [12]:
dataY = df['y']
dataX = df.drop('y', axis=1)

trainX, testX, trainY, testY = train_test_split(dataX, dataY, test_size=0.2, random_state=42)

# Train the model

In [13]:
model = SVC(kernel='rbf', gamma='auto')
model.fit(trainX, trainY)

SVC(gamma='auto')

# Predict value using test data

In [16]:
predictedY = model.predict(testX)

accuracy = accuracy_score(testY, predictedY)
print('Model Accuracy is:', str(math.floor(accuracy * 100)) + '%')

Model Accuracy is: 87%
